# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

**Menurut saya, decision tree adalah salah satu metode dalam supervised learning dalam ML yang digunakan untuk memprediksi dengan membagi data ke dalam cabang-cabang berdasarkan aturan tertentu.**

# Import Data & Libraries

In [32]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [33]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_vals(rows, col):
    unique_values = set([row[col] for row in rows])
    return unique_values

# contoh penggunaan
unique_vals(training_data, 0)

{'Green', 'Red', 'Yellow'}

In [34]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows, col):
    counts = {}
    for row in rows:
        label = row[col]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# contoh penggunaan
class_counts(training_data, 0)

{'Green': 1, 'Yellow': 2, 'Red': 2}

In [35]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    return isinstance(value, (int, float))

# contoh penggunaan
is_numeric(7)

True

In [36]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

# contoh penggunaan 1
print(Question(1, 3))

# contoh penggunaan 2
print(Question(0, 'Green'))

Is diameter >= 3?
Is color == Green?


In [37]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
  true_rows, false_rows = [], []
  for row in rows:
    if question.match(row):
      true_rows.append(row)
    else:
      false_rows.append(row)
  return true_rows, false_rows

# contoh penggunaan
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print(true_rows)
print(false_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [38]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
  counts = class_counts(rows, 2)
  impurity = 1
  for label in counts:
      prob_of_lbl = counts[label] / float(len(rows))
      impurity -= prob_of_lbl**2
  return impurity

# contoh penggunaan
gini(training_data)

0.6399999999999999

**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [39]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# contoh penggunaan
info_gain(true_rows, false_rows, gini(training_data))

0.37333333333333324

In [40]:
from os.path import split
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    best_gain = 0  # simpan IG tertinggi
    best_question = None
    current_uncertainty = gini(rows)  # impurity awal
    n_features = len(rows[0]) - 1     # jumlah kolom fitur (exclude label)

    for col in range(n_features):
        values = set([row[col] for row in rows])  # semua value unik di kolom ini

        for val in values:
            question = Question(col, val)

            # Splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # update best split
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

# contoh penggunaan
best_gain, best_question = find_best_split(training_data)
print("Best Gain:", best_gain)
print("Best Question:", best_question)

Best Gain: 0.37333333333333324
Best Question: Is diameter >= 3?


# Fungsi Decision Tree

In [41]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts(rows, 2)

leaf = Leaf(training_data)
print(leaf.predictions)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [42]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [43]:
# membangun decision tree secara rekursif
def build_tree(rows):

    gain, question = find_best_split(rows)

    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)

    true_branch = build_tree(true_rows)

    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

In [44]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # mencetak pertanyaan pada node saat ini
    print (spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

# contoh penggunaan
my_tree = build_tree(training_data)
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [45]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# contoh penggunaan
my_tree = build_tree(training_data)
print("Prediction for ['Green', 3, 'Apple']:", classify(['Green', 3, 'Apple'], my_tree))
print("Prediction for ['Red', 1, 'Grape']:", classify(['Red', 1, 'Grape'], my_tree))

Prediction for ['Green', 3, 'Apple']: {'Apple': 1}
Prediction for ['Red', 1, 'Grape']: {'Grape': 2}


In [46]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for label in counts.keys():
        probs[label] = str(int(counts[label] / total * 100)) + "%"
    return probs

# contoh penggunaan
print_leaf(classify(training_data[1], my_tree))

{'Apple': '50%', 'Lemon': '50%'}

# Predict Using Decision Tree

In [47]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
for row in testing_data:
    prediction = classify(row, my_tree)
    actual_label = row[-1]
    print("Actual: %s, Prediction: %s" % (actual_label, print_leaf(prediction)))

Actual: Apple, Prediction: {'Apple': '100%'}
Actual: Apple, Prediction: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape, Prediction: {'Grape': '100%'}
Actual: Grape, Prediction: {'Grape': '100%'}
Actual: Lemon, Prediction: {'Apple': '50%', 'Lemon': '50%'}
